# Metricizing LLMaAiTB-E

- Our Focus: Generation Quality
- Measurement Techniques: 
    - Vector Comparison
    - Human Preference Sample (A/B/C Testing)
    - ???
- Iterative documents to measure:
    - Slides (Teaching phase)
    - Questions (Testing phase)
- Resources for testing:
    - Human (From classes)
    - GPT4 (Generation)
    - LLMaAiTB-E (Teachabull)

## Vector Comparison
Embeddings: OpenAI’s text embeddings measure the relatedness of text strings.

# Slides

In [11]:
import openai
import json

from AITutor_Backend.src.BackendUtils.replicate_api import ReplicateAPI
USE_OPENAI = True

In [1]:
from pptx import Presentation

def extract_text_from_slide(slide):
    """Extracts title and content from a slide"""
    title = slide.shapes.title.text if slide.shapes.title else "No Title"
    content = []

    for shape in slide.shapes:
        if hasattr(shape, "text"):
            content.append(shape.text)

    return title, content

def read_pptx(file_path):
    """Reads a pptx file and prints the title and content of each slide"""
    prs = Presentation(file_path)

    for slide in prs.slides:
        title, content = extract_text_from_slide(slide)
        print(f"Title: {title}")
        print("Content:", "\n".join(content))
        print("-" * 40)

In [9]:
import PyPDF2

def read_pdf(file_path):
    """Reads a PDF file and prints the content of each page"""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for i in range(num_pages):
            page = reader.pages[i]
            text = page.extract_text()
            print(f"Page {i+1} Content:\n{text}")
            print("-" * 40)

# Replace 'your_presentation.pdf' with the path to your PDF file
read_pdf('/Users/fiorellaratti/Documents/IEEE-Teach-A-Bull/AITutor-Backend/Research/generation_data/slides/Expert/Teach-A-Bull_Slide1.pdf')


Page 1 Content:
Chapter#11: Main Pillars of  the 
Object Oriented Programming 
ParadigmAnasse Bari, Ph.D.Courant Institute of  Mathematical Sciences
Department of  Computer Science 
CS101 Introduction to Computer Science
----------------------------------------
Page 2 Content:
Objectives
❖Introducing the concepts of  Encapsulation and Inheritance
❖Learning the Super keyword, superclass methods and data fields
❖Introducing the concept of  polymorphism
❖Learning how to use interfaces and abstract classes in Java
❖Introducing the comparable java interface
❖Introducing casting and instance of
❖Understanding where to use an interface or an abstract class
2
----------------------------------------
Page 3 Content:
Main Pillars of  OOP
3

----------------------------------------
Page 4 Content:
Review Summary on OOP
❖Object -oriented Design: A problem -solving methodology that produces a solution to a problem in terms 
of  self -contained entities called objects
❖Object: A thing or entity that

In [13]:
prompt = ''' #Create a json object from slides. Slides will come in the following format:
        Page 1 Content:
        Chapter#11: Main Pillars of  the 
        Object Oriented Programming 
        ParadigmAnasse Bari, Ph.D.Courant Institute of  Mathematical Sciences
        Department of  Computer Science 
        CS101 Introduction to Computer Science
        ----------------------------------------
        Page 2 Content:
        Objectives
        ❖Introducing the concepts of  Encapsulation and Inheritance
        ❖Learning the Super keyword, superclass methods and data fields
        ❖Introducing the concept of  polymorphism
        ❖Learning how to use interfaces and abstract classes in Java
        ❖Introducing the comparable java interface
        ❖Introducing casting and instance of
        ❖Understanding where to use an interface or an abstract class
        2
        ----------------------------------------
        Page 3 Content:
        Main Pillars of  OOP
        3

        ----------------------------------------
        Page 4 Content:
        Review Summary on OOP
        ...
        ParadigmAnasse Bari, Ph.D.Courant Institute of  Mathematical Sciences
        Department of  Computer Science 
        CS101 Introduction to Computer Science


        #json object should have this format:

        json 
        { 
                "slides":[
                        {
                                \"Title\":\"Chapter#11: Main Pillars of  the Object Oriented Programming\", 
                                \"Description\": \"ParadigmAnasse Bari, Ph.D.Courant Institute of  Mathematical Sciences Department of  Computer Science CS101 Introduction to Computer Science\",
                                \"Latex\": []
                        },
                        {
                                \"Title\":\"Objectives\", 
                                \"Description\": \" -Introducing the concepts of  Encapsulation and Inheritance\n - Learning the Super keyword, superclass methods and data fields ❖Introducing the concept of  polymorphism ❖Learning how to use interfaces and abstract classes in Java ❖Introducing the comparable java interface ❖Introducing casting and instance of ❖Understanding where to use an interface or an abstract class\",
                                \"Latex\": []
                        },
                        {
                                \"Title\":\"Main Pillars of  OOP\", 
                                \"Description\": \"\",
                                \"Latex\": []
                        },
                ]
        
        '''


In [14]:
def request_output_from_llm(self, prompt, model: str):
    """Requests the Concept information from an LLM.

    Args:
        prompt: (str) - string to get passed to the model
        model: (str) - 

    Returns:
        _type_: _description_
    """
    client = openai.OpenAI() if USE_OPENAI else ReplicateAPI()
    
    if USE_OPENAI:
        response = client.chat.completions.create(
        model=model,
        messages=[
        {
        "role": "system",
        "content": prompt,
        },
        ],
        temperature=1,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        )

        return response.choices[0].message.content
    else:
        return client.get_output(prompt, " ")